<a href="https://colab.research.google.com/github/AmeerHamza-345/Q2/blob/main/Project_03_LangChain_Function_Tool_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U langchain-google-genai

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
llm:ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(
  model = "gemini-2.0-flash-exp",
  api_key = GOOGLE_API_KEY
)

In [ ]:
llm.invoke("hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-6c75cecf-fa66-40c2-bd60-393e95003fe3-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [ ]:
from langchain_core.tools import tool
@tool
def add(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b
@tool
def subtract(a: int, b: int) -> int:
    """Subtract two numbers"""
    return a - b
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b
@tool
def divide(a: int, b: int) -> int:
    """Divide two numbers"""
    return a / b
tools = [add, subtract, multiply, divide]

In [ ]:
llm.invoke("add  numbers 45 and 67 ")

AIMessage(content='45 + 67 = 112', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-ffc197f9-b4cd-4bd9-aca4-781b25716df6-0', usage_metadata={'input_tokens': 12, 'output_tokens': 12, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}})

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langchain_core.messages import HumanMessage
query = "What is 3 * 12? Also, what is 11 + 49?"
messages = [HumanMessage(query)]
display(messages)

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={})]

In [ ]:
ai_message = llm_with_tools.invoke(messages)
messages.append(ai_message)
messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 11.0, "b": 49.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-9d230788-1bb1-4c60-bf34-ce0bc5204203-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': '0b283a2e-d930-4e59-a321-8d1ab539ba1d', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': '3b053d7f-974b-48eb-9872-63101671aba4', 't

In [ ]:
llm_with_tools.invoke("messages")

AIMessage(content='What would you like to calculate?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-a9d19a83-1279-4efc-9bfb-304033636f3f-0', usage_metadata={'input_tokens': 179, 'output_tokens': 8, 'total_tokens': 187, 'input_token_details': {'cache_read': 0}})

In [ ]:
ai_message.tool_calls

[{'name': 'multiply',
  'args': {'a': 3.0, 'b': 12.0},
  'id': '0b283a2e-d930-4e59-a321-8d1ab539ba1d',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11.0, 'b': 49.0},
  'id': '3b053d7f-974b-48eb-9872-63101671aba4',
  'type': 'tool_call'}]

In [ ]:
for tool_call in ai_message.tool_calls:
    selected_tool = {
        "add": add,
        "subtract": subtract,
        "multiply": multiply,
        "divide": divide,
    }[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    display(tool_msg)
    messages.append(tool_msg)

ToolMessage(content='36', name='multiply', tool_call_id='0b283a2e-d930-4e59-a321-8d1ab539ba1d')

ToolMessage(content='60', name='add', tool_call_id='3b053d7f-974b-48eb-9872-63101671aba4')

In [ ]:
display(messages)

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 11.0, "b": 49.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-9d230788-1bb1-4c60-bf34-ce0bc5204203-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': '0b283a2e-d930-4e59-a321-8d1ab539ba1d', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': '3b053d7f-974b-48eb-9872-63101671aba4', 't

In [ ]:
response = llm_with_tools.invoke(messages)
response

AIMessage(content='3 multiplied by 12 is 36. 11 plus 49 is 60.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-4745a163-fc94-4a8d-8116-5570d78d2b57-0', usage_metadata={'input_tokens': 269, 'output_tokens': 24, 'total_tokens': 293, 'input_token_details': {'cache_read': 0}})

In [ ]:
from IPython.display import Markdown
Markdown(response.content)

3 multiplied by 12 is 36. 11 plus 49 is 60.